# Add Student

## 1. Add users to JupyterHub

[Add users to JupyterHub first](https://tljh.servehttp.com/hub/admin). Click on "Add Users".

## 2. Generate list of students to import

In [ ]:
# Create list to import as import_list
# This script reads students from students.csv
# students.csv should have comma-separated student data
# in the following format:
# id,last_name,first_name,email

import_list = []
with open('students.csv','rt') as f:
    header = f.readline().strip().split(',')
    for line in f:
        data = line.strip().split(',')
        student = dict(zip(header,data))
        student['id'] = student['id'].lower()
        import_list.append(student)

## 3. Importing students into nbgrader database

You will need to [generate the student list](#Generate-student-list) first.

In [ ]:
! nbgrader db student import students.csv

# TODO: Refactor to modularise api and gb import

import subprocess
# Use nbgrader api to verify that users have been imported into nbgrader
from nbgrader.apps import NbGraderAPI
from nbgrader.api import Gradebook
from traitlets.config import Config
from datetime import datetime

# Use python-dotenv to load environment variables for better security
from dotenv import load_dotenv
load_dotenv()

COURSE_ROOT = os.environ.get("course-root")
COURSE_ID = os.environ.get("course-id")

# create a custom config object to specify options for nbgrader API
config = Config()
config.CourseDirectory.root = COURSE_ROOT
config.CourseDirectory.course_id = COURSE_ID

api = NbGraderAPI(config=config)
gb = Gradebook(f'sqlite://{COURSE_ROOT}/gradebook.db')

student_list = [student['id'] for student in api.get_students()]

for orig_student in import_list:
    from copy import copy
    try:
        student = orig_student.copy()
        student_id = student.pop('id')
        tljh_id = f"jupyter-{student_id}"
        gb.update_or_create_student(tljh_id, **student)
    except Exception as e:
        raise
    else:
        print(f"[DONE] {tljh_id} added")
        if student_id in student_list and tljh_id in student_list:
            gb.remove_student(student_id)
#             subprocess.call(["nbgrader","db","student","remove",student_id])
            print(f"[DUPE] {student_id} removed")
# Cleanup
del student_list

# Student list

See [Manage Students](../../formgrader/manage_students) in Formgrader.

# Remove student

In [ ]:
import os
from nbgrader.apps import NbGraderAPI
from nbgrader.api import Gradebook, MissingEntry
from traitlets.config import Config

# Use python-dotenv to load environment variables for better security
from dotenv import load_dotenv
load_dotenv()

COURSE_ROOT = os.environ.get("course-root")
COURSE_ID = os.environ.get("course-id")

# create a custom config object to specify options for nbgrader API
config = Config()
config.CourseDirectory.root = COURSE_ROOT
config.CourseDirectory.course_id = COURSE_ID

gb = Gradebook(f'sqlite://{COURSE_ROOT}/gradebook.db')
api = NbGraderAPI(config=config)

def delete_from_system(student_id):
    import subprocess
    status_code = subprocess.call(["sudo","userdel",s_id])
    if status_code == 6:
        print(f'[ ERROR ] User {s_id} does not exist in system.')
    status_code = subprocess.call(["sudo","groupdel",s_id])
    if status_code == 6:
        print(f'[ ERROR ] Group {s_id} does not exist in system.')
        
def remove_from_nbgrader(student_id):
    try:
        gb.remove_student(s_id)
    except Exception as e:
        raise
    else:
        print(f'[SUCCESS] User {s_id} successfully deleted.')

def delete_files(student_id):
    import subprocess
    # TODO: use pathlib for more reliable path joining
    dirs = {'submitted': f'{COURSE_ROOT}/submitted/',
            'autograded': f'{COURSE_ROOT}/autograded/',
            'feedback': f'{COURSE_ROOT}/feedback/',
              }
    status = []
    # TODO: Use Python shutil for removal; rm -rf not recommended
    for dir_,path in dirs.items():
        if subprocess.call(["sudo","rm","-R",f"{path}{s_id}"]) == 0:
            status.append(dir_)
    if any(status):
        print(f"[SUCCESS] Deleted {s_id} from {', '.join(status)}")
    else:
        print(f"[ ERROR ] {s_id} does not exist in nbgrader directories.")
        
def main_menu(students):
    print('\n# Student list')
    for idx,s in enumerate(students):
        print(f'{idx:2}. {s["id"]}')
    selected = input('Choose student by id (leave blank to quit): ')
    if selected == '':
        return None
    else:
        return int(selected)

def student_info(students,idx):
    # menu options
    menu = {'remove': 'r',
            'main': 'm',
            'delete': 'd',
            }
    this = api.get_student(students[idx]['id'])
    klen = max([len(k) for k in this.keys()])
    print(f'\n## Info for {this["first_name"]}')
    for k,v in this.items():
        print(f'{k:{klen}}: {v}')
    ret1 = input(f"'{menu['remove']}' to remove user, \
'{menu['delete']}' to delete user, \
'{menu['main']}' to return to main menu, \
or any key to quit: ")
    ret2 = this['id'] if ret1.lower() == menu['remove'] else None
    return ret1,ret2

In [ ]:
quit = False
while not quit:
    students = api.get_students()
    s_idx = main_menu(students)
    if s_idx is None:
        break
    select,s_id = student_info(students,s_idx)
    if select == 'm':
        quit = False
    elif select == 'r':
        remove_from_nbgrader(s_id)
    elif select == 'd':
        try:
            remove_from_nbgrader(s_id)
        except MissingEntry:
            print(f"Student {s_id} not found in database.")
        finally:
            delete_files(s_id)
    else:
        quit = True